Download dataset

In [7]:
!git lfs install

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log


In [8]:
!git clone git@hf.co:datasets/mesolitica/pseudolabel-malaya-speech-stt-train-whisper-large-v3-timestamp/tree/main
!git clone git@hf.co:datasets/mesolitica/pseudolabel-imda-large-v3-timestamp
!git clone git@hf.co:datasets/mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3-timestamp
!git clone git@hf.co:datasets/mesolitica/pseudolabel-indonesian-large-v3-timestamp
!git clone git@hf.co:datasets/mesolitica/pseudolabel-nusantara-large-v3-timestamp

Cloning into 'main'...
The authenticity of host 'hf.co (34.204.155.59)' can't be established.
ED25519 key fingerprint is SHA256:dVjzGIdV7d6cwKIeZiCoRMa2gMvSKfGZAvHf4gMiMao.
This key is not known by any other names.
Are you sure you want to continue connecting (yes/no/[fingerprint])? ^C


In [2]:
import json
import os
from tqdm import tqdm
from glob import glob
from streaming import MDSWriter

columns = {
    'audio_filename': 'str',
    'text': 'str',
}
hashes = 'sha1', 'xxh64'

Define files (add all from provided datasets)

In [3]:
files = [
    'prepared-pseudolabel-malaya.jsonl',
    'prepared-pseudolabel-malaya-original.jsonl',
    'prepared-imda-ms.jsonl',
    'prepared-imda.jsonl',
    'prepared-pseudolabel-original.jsonl'
    'prepared-pseudolabel-original.jsonl',
    'prepared-pseudolabel.jsonl',
    'prepared-indonesian-en.jsonl',
    'prepared-indonesian-ms.jsonl',
    'prepared-indonesian-original.jsonl',
    'prepared-nusantara-original.jsonl',
    'prepared-nusantara.jsonl'
]

In [4]:
!wc -l {files[0]}

    6274 /Users/halimshukor/Documents/github-projects/whisper-distill-finetune/nusantara-data/prepared-nusantara-original.jsonl


In [5]:
!wc -l {files[1]}

 3085595 /Users/halimshukor/Documents/github-projects/whisper-distill-finetune/malaysian-youtube-data/prepared-pseudolabel.jsonl


In [ ]:
with MDSWriter(out='mosaic-whisper', columns=columns, compression=None, hashes=hashes) as out:
    for f in files:
        folder = os.path.split(f)[0]
        with open(f) as fopen:
            for l in tqdm(fopen):
                l = json.loads(l)
                for r in l:
                    audio_filename = os.path.join(folder, r['audio_filename'])
                    audio_filename = os.path.abspath(audio_filename)
                    t = r['text'].strip()
                    before = t
                    t = ' '.join([w for w in t.split() if len(w) > 1]).strip()
                    if len(t) < 3:
                        continue
                    d = {
                        'text': t,
                        'audio_filename': audio_filename,
                    }
                    out.write(d)


Streaming from dataset

In [ ]:
from torch.utils.data import DataLoader
from streaming import StreamingDataset, Stream


local = '/tmp/path-to-dataset'

hugginface_remote_list = ["hf://mesolitica/pseudolabel-malaya-speech-stt-train-whisper-large-v3-timestamp",
                          "hf://mesolitica/pseudolabel-imda-large-v3-timestamp",
                          "hf://mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3-timestamp",
                          "hf://mesolitica/pseudolabel-indonesian-large-v3-timestamp",
                          "hf://mesolitica/pseudolabel-nusantara-large-v3-timestamp"]

streams = [Stream(remote=remote, local=local) for remote in hugginface_remote_list]

dataset = StreamingDataset(
  streams=streams,
  samples_per_epoch=1e8,
)

row = dataset[0]
text = row['text']
audio = row['audio_filename']


dataloader = DataLoader(dataset)